In [1]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable

import numpy as np
import jax
import jax.numpy as jnp
import jax.lax as lax
from jaxtyping import Array, Float, Int, PRNGKeyArray
import aeon

from features.sig_trp import SigVanillaTensorizedRandProj, SigRBFTensorizedRandProj
from features.sig import SigTransform, LogSigTransform
from features.base import TimeseriesFeatureTransformer, TabularTimeseriesFeatures, RandomGuesser
from features.sig_neural import RandomizedSignature
from utils import print_name, print_shape

jax.config.update('jax_platform_name', 'gpu') # Used to set the platform (cpu, gpu, etc.)
np.set_printoptions(precision=3, threshold=5) # Print options

/home/nikita/Code/zephyrox/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# aoen toolkit

In [2]:
# Print the different datasets
from aeon.datasets.tsc_datasets import multivariate, univariate, univariate_equal_length
from aeon.datasets import load_classification

def get_aeon_dataset(
        dataset_name:str, 
        extract_path = "/home/nikita/hdd/Data/TSC/"
        ):
    """Loads a dataset from the UCR/UEA archive using 
    the aeon library.

    Args:
        dataset_name (str): Name of the dataset

    Returns:
        Tuple: 4-tuple of the form (X_train, y_train, X_test, y_test)
    """
    X_train, y_train = load_classification(dataset_name, split="train", extract_path=extract_path)
    X_test, y_test = load_classification(dataset_name, split="test", extract_path=extract_path)

    return X_train.transpose(0,2,1), y_train, X_test.transpose(0,2,1), y_test

univariate

{'ACSF1',
 'Adiac',
 'AllGestureWiimoteX',
 'AllGestureWiimoteY',
 'AllGestureWiimoteZ',
 'ArrowHead',
 'BME',
 'Beef',
 'BeetleFly',
 'BirdChicken',
 'CBF',
 'Car',
 'Chinatown',
 'ChlorineConcentration',
 'CinCECGTorso',
 'Coffee',
 'Computers',
 'CricketX',
 'CricketY',
 'CricketZ',
 'Crop',
 'DiatomSizeReduction',
 'DistalPhalanxOutlineAgeGroup',
 'DistalPhalanxOutlineCorrect',
 'DistalPhalanxTW',
 'DodgerLoopDay',
 'DodgerLoopGame',
 'DodgerLoopWeekend',
 'ECG200',
 'ECG5000',
 'ECGFiveDays',
 'EOGHorizontalSignal',
 'EOGVerticalSignal',
 'Earthquakes',
 'ElectricDevices',
 'EthanolLevel',
 'FaceAll',
 'FaceFour',
 'FacesUCR',
 'FiftyWords',
 'Fish',
 'FordA',
 'FordB',
 'FreezerRegularTrain',
 'FreezerSmallTrain',
 'Fungi',
 'GestureMidAirD1',
 'GestureMidAirD2',
 'GestureMidAirD3',
 'GesturePebbleZ1',
 'GesturePebbleZ2',
 'GunPoint',
 'GunPointAgeSpan',
 'GunPointMaleVersusFemale',
 'GunPointOldVersusYoung',
 'Ham',
 'HandOutlines',
 'Haptics',
 'Herring',
 'HouseTwenty',
 'Inli

In [3]:
#from aeon.transformations.collection.convolution_based import MiniRocketMultivariate
from preprocessing.timeseries_augmentation import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from aeon.classification.sklearn import RotationForestClassifier
#from sklearn.linear_model import RidgeCV
from sklearn.metrics import accuracy_score
import time

def train_and_test(
        dataset:str,
        transformer:TimeseriesFeatureTransformer,
        apply_augmentation:bool=True,
    ):
    train_X, train_y, test_X, test_y = get_aeon_dataset(dataset)
    train_X, test_X = normalize_streams(train_X, test_X, max_T=1000)
    print_name(train_X)
    print_name(test_X)
    # augment data
    train_X = lax.stop_gradient(jnp.array(train_X))
    test_X  = lax.stop_gradient(jnp.array(test_X))
    if apply_augmentation:
        train_X = add_basepoint_zero(train_X)
        train_X = augment_time(train_X)
        test_X  = add_basepoint_zero(test_X)
        test_X  = augment_time(test_X)

    # fit transformer
    t0 = time.time()
    transformer.fit(train_X)
    train_X = np.array(transformer.transform(train_X))
    test_X = np.array(transformer.transform(test_X))
    train_X, test_X = normalize_mean_std_traindata(train_X, test_X)
    t1 = time.time()
    print_name(train_X)
    print_name(test_X)
    print(f"Time to transform: {t1-t0} seconds")

    # train classifier      
    clf = RotationForestClassifier()
    clf.fit(train_X, train_y)
    t2 = time.time()
    print(f"Time to fit classifier on train: {t2-t1} seconds")

    # predict
    pred = clf.predict(test_X)
    acc = accuracy_score(test_y, pred)
    t3 = time.time()
    print(f"Time to predict: {t3-t2} seconds")
    print(f"{acc} accuracy for {transformer}")

In [ ]:
train_and_test(
    dataset="Adiac",
    transformer=SigVanillaTensorizedRandProj(
        jax.random.PRNGKey(999),
        n_features= 128,
        trunc_level=5,
        max_batch=2000,
        )
    )
# 0.6086956521739131 accuracy for SigVanillaTensorizedRandProj(max_batch=10000, n_features=256,
#                              seed=Array([  0, 999], dtype=uint32),
#                              trunc_level=5)

In [ ]:
train_and_test(
    dataset="Adiac",
    transformer=SigRBFTensorizedRandProj(
        jax.random.PRNGKey(999),
        n_features= 256,
        trunc_level=3,
        rbf_dimension=1000,
        max_batch=10000,
        )
    )
# 0.6445012787723785 accuracy for SigRBFTensorizedRandProj(max_batch=10000, n_features=256, rbf_dimension=1000,
#                          rff_max_batch=10000,
#                          rff_seed=Array([4116651765, 1982142802], dtype=uint32),
#                          trp_seed=Array([3655788082, 2541180754], dtype=uint32))

In [ ]:
train_and_test(
    dataset="Adiac",
    transformer=TabularTimeseriesFeatures(),
    apply_augmentation=False
    )
# 0.7902813299232737 accuracy for TabularTimeseriesFeatures()

In [ ]:
train_and_test(
    dataset="Adiac",
    transformer=SigTransform(trunc_level=5),
    ) 
# 0.5498721227621484 accuracy for SigTransform(trunc_level=5)

In [ ]:
train_and_test(
    dataset="Adiac",
    transformer=LogSigTransform(trunc_level=5),
    ) 
# 0.4884910485933504 accuracy for LogSigTransform(trunc_level=5)


In [ ]:
train_and_test(
    dataset="Adiac",
    transformer=RandomizedSignature(
        jax.random.PRNGKey(999),
        n_features= 128,
        max_batch=10000,
        )
    )
# 0.27365728900255754 accuracy for RandomizedSignature(max_batch=10000, n_features=128,
#                     seed=Array([  0, 999], dtype=uint32))

In [ ]:
train_and_test(
    dataset="Adiac",
    transformer=RandomGuesser(
        jax.random.PRNGKey(999),
        n_features= 64,
        )
    )
# 0.03324808184143223 accuracy for RandomGuesser(n_features=64,
#                     seed=Array([1508125853,  174035561], dtype=uint32))

# Ridge

In [4]:
from sklearn.linear_model import RidgeClassifierCV

def train_and_test_ridge(
        dataset:str,
        transformer:TimeseriesFeatureTransformer,
        apply_augmentation:bool=True,
    ):
    train_X, train_y, test_X, test_y = get_aeon_dataset(dataset)
    train_X, test_X = normalize_streams(train_X, test_X, max_T=1000)
    print_shape(train_X)
    print_shape(test_X)
    # augment data
    train_X = lax.stop_gradient(jnp.array(train_X))
    test_X  = lax.stop_gradient(jnp.array(test_X))
    if apply_augmentation:
        train_X = add_basepoint_zero(train_X)
        train_X = augment_time(train_X)
        test_X  = add_basepoint_zero(test_X)
        test_X  = augment_time(test_X)

    # fit transformer
    t0 = time.time()
    transformer.fit(train_X)
    feat_train_X = np.array(transformer.transform(train_X))
    feat_test_X = np.array(transformer.transform(test_X))
    print("Before normalization:")
    print_name(feat_test_X)
    feat_train_X, feat_test_X = normalize_mean_std_traindata(feat_train_X, feat_test_X)
    t1 = time.time()
    print("After normalization:")
    print_name(feat_test_X)
    print(f"Time to transform: {t1-t0} seconds")

    # train classifier      
    clf = RidgeClassifierCV(alphas=np.logspace(-5, -2, 50))
    clf.fit(feat_train_X, train_y)
    t2 = time.time()
    print(f"Chosen alpha: {clf.alpha_}")
    print(f"Time to fit classifier on train: {t2-t1} seconds")

    # predict
    pred = clf.predict(feat_test_X)
    print(pred)
    acc = accuracy_score(test_y, pred)
    t3 = time.time()
    print(f"Time to predict: {t3-t2} seconds")
    print(f"{acc} accuracy for {transformer}")

In [6]:
train_and_test_ridge(
    dataset="MiddlePhalanxTW",
    transformer=SigVanillaTensorizedRandProj(
        jax.random.PRNGKey(999),
        n_features= 1000,
        trunc_level=5,
        max_batch=10,
        )
    )
# 0.5 accuracy for SigVanillaTensorizedRandProj(max_batch=20, n_features=1000,
#                              seed=Array([  0, 999], dtype=uint32),
#                              trunc_level=5)

(399, 80, 1) train_X 

(154, 80, 1) test_X 

Before normalization:
(154, 1000) feat_test_X
[[ 1.061e-02 -2.562e-01 -2.420e-03 ... -8.838e-02  2.902e-01 -3.519e-02]
 [ 9.577e-03 -2.312e-01 -2.657e-03 ... -7.685e-02  2.541e-01 -3.864e-02]
 [ 4.190e-02 -5.437e-01  1.797e-04 ...  7.499e-02  6.753e-01  5.735e-01]
 ...
 [ 1.290e-03 -6.411e-02 -1.009e-03 ... -2.262e-02  7.382e-02  3.431e-02]
 [ 3.358e-03 -1.608e-01 -6.299e-03 ... -2.332e-02  2.818e-01 -2.035e-02]
 [ 4.669e-03 -1.223e-01 -2.381e-03 ... -2.654e-02  2.116e-01  1.806e-02]] 

After normalization:
(154, 1000) feat_test_X
[[-0.356  0.324  0.222 ...  0.019 -0.348 -0.419]
 [-0.38   0.356  0.199 ...  0.054 -0.376 -0.427]
 [ 0.389 -0.044  0.467 ...  0.527 -0.055  1.01 ]
 ...
 [-0.578  0.569  0.355 ...  0.223 -0.513 -0.256]
 [-0.529  0.446 -0.144 ...  0.221 -0.355 -0.384]
 [-0.497  0.495  0.226 ...  0.211 -0.408 -0.294]] 

Time to transform: 1.8579745292663574 seconds
Chosen alpha: 0.01
Time to fit classifier on train: 0.1330082416534423

In [7]:
train_and_test_ridge(
    dataset="MiddlePhalanxTW",
    transformer=SigRBFTensorizedRandProj(
        jax.random.PRNGKey(999),
        n_features= 1000,
        trunc_level=5,
        rbf_dimension=1000,
        max_batch=10,
        rff_max_batch=10000,
        )
    )
# 0.5333333333333333 accuracy for SigRBFTensorizedRandProj(max_batch=10, n_features=1000, rbf_dimension=1000,
#                          rff_max_batch=10000,
#                          seed=Array([  0, 999], dtype=uint32), trunc_level=5)

(399, 80, 1) train_X 

(154, 80, 1) test_X 

Before normalization:
(154, 1000) feat_test_X
[[ 1.074e-08  7.490e-08 -1.187e-09 ... -3.183e-09 -8.982e-08 -7.769e-10]
 [ 3.683e-09  8.412e-08 -2.039e-09 ... -3.544e-09 -9.572e-08  2.917e-09]
 [-4.099e-08 -1.939e-07 -1.057e-09 ...  1.044e-08 -1.792e-07  1.398e-08]
 ...
 [ 6.406e-09 -9.757e-09  7.331e-11 ...  1.817e-09  1.201e-08 -9.141e-10]
 [ 3.085e-10  2.647e-08  9.507e-10 ...  7.379e-10 -1.846e-08 -1.871e-09]
 [-4.036e-09  1.898e-09  2.575e-09 ... -6.353e-10 -1.053e-08  2.078e-10]] 

After normalization:
(154, 1000) feat_test_X
[[ 2.847e-01  3.214e-01 -4.687e-01 ... -8.391e-02  2.525e-01 -2.549e-01]
 [ 9.261e-02  3.934e-01 -5.438e-01 ... -9.696e-02  2.077e-01 -1.467e-01]
 [-1.123e+00 -1.777e+00 -4.572e-01 ...  4.088e-01 -4.273e-01  1.772e-01]
 ...
 [ 1.667e-01 -3.396e-01 -3.575e-01 ...  9.696e-02  1.027e+00 -2.590e-01]
 [ 7.885e-04 -5.673e-02 -2.801e-01 ...  5.792e-02  7.953e-01 -2.870e-01]
 [-1.174e-01 -2.486e-01 -1.367e-01 ...  8.248e-0

In [8]:
train_and_test_ridge(
    dataset="MiddlePhalanxTW",
    transformer=TabularTimeseriesFeatures(),
    apply_augmentation=False
    )
# 0.8666666666666667 accuracy for TabularTimeseriesFeatures()

(399, 80, 1) train_X 

(154, 80, 1) test_X 

Before normalization:
(154, 80) feat_test_X
[[ 0.612  0.75   0.92  ... -0.884 -0.995 -1.041]
 [ 0.841  0.784  0.994 ... -0.875 -0.989 -1.037]
 [-1.778 -1.752 -1.924 ...  1.072  1.315  1.345]
 ...
 [-0.902 -0.576 -0.701 ...  0.618  0.659  0.672]
 [ 0.259  0.257 -0.427 ... -0.319 -0.16   0.2  ]
 [ 0.084 -0.114  0.005 ...  1.089  0.925  0.33 ]] 

After normalization:
(154, 80) feat_test_X
[[ 0.612  0.75   0.92  ... -0.884 -0.995 -1.041]
 [ 0.841  0.784  0.994 ... -0.875 -0.989 -1.037]
 [-1.778 -1.752 -1.924 ...  1.072  1.315  1.345]
 ...
 [-0.902 -0.576 -0.701 ...  0.618  0.659  0.672]
 [ 0.259  0.257 -0.427 ... -0.319 -0.16   0.2  ]
 [ 0.084 -0.114  0.005 ...  1.089  0.925  0.33 ]] 

Time to transform: 0.40603160858154297 seconds
Chosen alpha: 0.01
Time to fit classifier on train: 0.0650479793548584 seconds
['8' '8' '5' ... '4' '4' '3']
Time to predict: 0.001054525375366211 seconds
0.5324675324675324 accuracy for TabularTimeseriesFeatures()


In [9]:
train_and_test_ridge(
    dataset="MiddlePhalanxTW",
    transformer=SigTransform(trunc_level=5),
    )
# 0.6 accuracy for SigTransform(trunc_level=5)

(399, 80, 1) train_X 

(154, 80, 1) test_X 

Before normalization:
(154, 62) feat_test_X
[[-1.041e+00  1.000e+00  5.421e-01 ... -9.648e-04 -3.947e-02  8.333e-03]
 [-1.037e+00  1.000e+00  5.375e-01 ... -1.693e-03 -4.009e-02  8.333e-03]
 [ 1.345e+00  1.000e+00  9.044e-01 ...  1.256e-02  4.485e-02  8.333e-03]
 ...
 [ 6.715e-01  1.000e+00  2.255e-01 ...  4.368e-03  1.684e-02  8.333e-03]
 [ 2.000e-01  1.000e+00  2.001e-02 ... -1.211e-02  2.167e-02  8.333e-03]
 [ 3.299e-01  1.000e+00  5.443e-02 ...  3.076e-03  5.803e-03  8.333e-03]] 

After normalization:
(154, 62) feat_test_X
[[-1.041e+00  0.000e+00  1.017e-01 ... -1.239e-01 -1.039e+00  2.429e-07]
 [-1.037e+00  0.000e+00  9.057e-02 ... -2.192e-01 -1.055e+00  2.429e-07]
 [ 1.345e+00  0.000e+00  9.757e-01 ...  1.645e+00  1.180e+00  2.429e-07]
 ...
 [ 6.715e-01  0.000e+00 -6.624e-01 ...  5.737e-01  4.429e-01  2.429e-07]
 [ 2.000e-01  0.000e+00 -1.158e+00 ... -1.581e+00  5.699e-01  2.429e-07]
 [ 3.299e-01  0.000e+00 -1.075e+00 ...  4.046e-01  1

In [10]:
train_and_test_ridge(
    dataset="MiddlePhalanxTW",
    transformer=LogSigTransform(trunc_level=5),
    )
# 0.5 accuracy for LogSigTransform(trunc_level=5)

(399, 80, 1) train_X 

(154, 80, 1) test_X 

Before normalization:
(154, 14) feat_test_X
[[-1.041e+00  1.000e+00  5.851e-01 ...  4.342e-03 -1.079e-02  3.360e-04]
 [-1.037e+00  1.000e+00  5.997e-01 ...  5.491e-03 -1.306e-02  5.098e-04]
 [ 1.345e+00  1.000e+00 -7.598e-01 ...  3.752e-03 -6.248e-03 -1.022e-03]
 ...
 [ 6.715e-01  1.000e+00 -2.615e-01 ... -1.954e-03  3.862e-03 -3.209e-05]
 [ 2.000e-01  1.000e+00 -2.569e-01 ...  5.634e-03 -1.246e-02 -4.546e-04]
 [ 3.299e-01  1.000e+00 -1.331e-01 ... -1.740e-03  4.873e-03  4.089e-04]] 

After normalization:
(154, 14) feat_test_X
[[-1.041  0.     0.995 ...  0.357 -0.438  0.422]
 [-1.037  0.     1.02  ...  0.535 -0.591  0.639]
 [ 1.345  0.    -1.293 ...  0.265 -0.13  -1.273]
 ...
 [ 0.672  0.    -0.445 ... -0.622  0.554 -0.038]
 [ 0.2    0.    -0.437 ...  0.558 -0.551 -0.565]
 [ 0.33   0.    -0.227 ... -0.588  0.622  0.513]] 

Time to transform: 1.0599665641784668 seconds
Chosen alpha: 0.01
Time to fit classifier on train: 0.007568836212158203 s

In [12]:
train_and_test_ridge(
    dataset="MiddlePhalanxTW",
    transformer=RandomizedSignature(
        jax.random.PRNGKey(999),
        n_features= 1000,
        max_batch=10,
        )
    )
# 0.4 accuracy for RandomizedSignature(max_batch=10, n_features=1000,
#                     seed=Array([  0, 999], dtype=uint32))

(399, 80, 1) train_X 

(154, 80, 1) test_X 

Before normalization:
(154, 1000) feat_test_X
[[ 1.159e-03  3.374e-02  3.414e-02 ... -5.736e-02 -3.409e-02 -4.219e-02]
 [ 5.508e-02  3.569e-02  6.042e-02 ... -4.170e-02 -3.563e-02  1.866e-02]
 [ 4.207e-02 -1.737e-02  2.756e-02 ... -4.024e-02  1.737e-02  1.737e-02]
 ...
 [-1.327e-03  2.026e-04  2.341e-02 ...  1.607e-03 -1.196e-02  2.012e-03]
 [ 3.728e-01  3.480e-01 -2.131e-01 ...  3.729e-01  3.728e-01  3.730e-01]
 [ 5.748e-01 -6.080e-01 -5.824e-01 ... -5.590e-01  6.080e-01 -5.641e-01]] 

After normalization:
(154, 1000) feat_test_X
[[-0.129  0.162  0.023 ... -0.305 -0.206 -0.24 ]
 [ 0.262  0.176  0.224 ... -0.19  -0.217  0.196]
 [ 0.167 -0.214 -0.028 ... -0.179  0.161  0.187]
 ...
 [-0.148 -0.085 -0.059 ...  0.129 -0.048  0.077]
 [ 2.567  2.475 -1.873 ...  2.865  2.692  2.733]
 [ 4.033 -4.561 -4.704 ... -4.001  4.368 -3.976]] 

Time to transform: 4.240599632263184 seconds
Chosen alpha: 0.01
Time to fit classifier on train: 0.10120058059692383

In [17]:
train_and_test_ridge(
    dataset="MiddlePhalanxTW",
    transformer=RandomGuesser()
    )
# 0.43333333333333335 accuracy for RandomGuesser(seed=Array([2384771982, 3928867769], dtype=uint32))

(399, 80, 1) train_X 

(154, 80, 1) test_X 

Before normalization:
(154, 512) feat_test_X
[[ 0.82  -0.336 -0.216 ...  1.164 -2.067  0.598]
 [ 1.1    0.063 -0.833 ...  1.215 -0.908 -2.285]
 [ 2.152  0.827 -0.355 ... -0.049 -1.399  0.492]
 ...
 [ 1.179  0.623  0.513 ...  0.802 -0.41  -0.432]
 [-0.699  0.53  -1.237 ...  0.483 -0.844 -0.101]
 [ 1.321 -1.496  0.828 ...  0.683 -0.657  0.643]] 

After normalization:
(154, 512) feat_test_X
[[ 0.898 -0.229 -0.165 ...  1.173 -1.884  0.583]
 [ 1.183  0.148 -0.807 ...  1.225 -0.773 -2.276]
 [ 2.259  0.871 -0.31  ... -0.07  -1.243  0.478]
 ...
 [ 1.265  0.678  0.593 ...  0.802 -0.297 -0.438]
 [-0.654  0.59  -1.227 ...  0.475 -0.713 -0.111]
 [ 1.41  -1.326  0.922 ...  0.68  -0.533  0.627]] 

Time to transform: 0.015050888061523438 seconds
Chosen alpha: 0.01
Time to fit classifier on train: 0.14815235137939453 seconds
['8' '3' '4' ... '5' '8' '3']
Time to predict: 0.0011222362518310547 seconds
0.18181818181818182 accuracy for RandomGuesser()


In [13]:
from sklearn.linear_model import RidgeClassifierCV
from aeon.transformations.collection.convolution_based import Rocket, MultiRocket, MiniRocket

def train_and_test_ridge_ROCKETS(
        dataset:str,
        transformer,
        apply_augmentation:bool=True,
    ):
    train_X, train_y, test_X, test_y = get_aeon_dataset(dataset)

    print_shape(train_X)
    print_shape(test_X)
    # augment data
    train_X = train_X.transpose(0,2,1)
    test_X  = test_X.transpose(0,2,1)
    if apply_augmentation:
        train_X, test_X = normalize_streams(train_X, test_X, max_T=1000)
        train_X = np.array(add_basepoint_zero(train_X))
        test_X  = np.array(add_basepoint_zero(test_X))

    # fit transformer
    t0 = time.time()
    transformer.fit(train_X)
    feat_train_X = np.array(transformer.transform(train_X))
    feat_test_X = np.array(transformer.transform(test_X))
    print("Before normalization:")
    print_name(feat_test_X)
    feat_train_X, feat_test_X = normalize_mean_std_traindata(feat_train_X, feat_test_X)
    t1 = time.time()
    print("After normalization:")
    print_name(feat_test_X)
    print(f"Time to transform: {t1-t0} seconds")

    # train classifier      
    clf = RidgeClassifierCV(alphas=np.logspace(-3, 3, 100))
    clf.fit(feat_train_X, train_y)
    t2 = time.time()
    print(f"Chosen alpha: {clf.alpha_}")
    print(f"Time to fit classifier on train: {t2-t1} seconds")

    # predict
    pred = clf.predict(feat_test_X)
    print(pred)
    acc = accuracy_score(test_y, pred)
    t3 = time.time()
    print(f"Time to predict: {t3-t2} seconds")
    print(f"{acc} accuracy for {transformer}")

In [16]:
train_and_test_ridge_ROCKETS(
    dataset="MiddlePhalanxTW",
    transformer=Rocket(num_kernels=5000),
    apply_augmentation=True
    )
# 0.8 accuracy for Rocket(num_kernels=500)

(399, 80, 1) train_X 

(154, 80, 1) test_X 

Before normalization:
(154, 10000) feat_test_X
[[0.375 3.293 0.597 ... 0.91  0.517 8.766]
 [0.412 3.574 0.556 ... 0.91  0.483 9.259]
 [0.412 3.736 0.528 ... 0.91  0.45  7.46 ]
 ...
 [0.412 3.476 0.611 ... 0.91  0.483 6.229]
 [0.412 2.587 0.556 ... 0.91  0.467 6.351]
 [0.375 3.161 0.569 ... 0.91  0.583 7.168]] 

After normalization:
(154, 10000) feat_test_X
[[-0.479 -0.024  0.41  ... -1.     1.012  0.499]
 [ 0.493  0.489 -0.494 ... -1.     0.26   0.862]
 [ 0.493  0.788 -1.096 ... -1.    -0.492 -0.464]
 ...
 [ 0.493  0.31   0.711 ... -1.     0.26  -1.371]
 [ 0.493 -1.319 -0.494 ... -1.    -0.116 -1.281]
 [-0.479 -0.267 -0.193 ... -1.     2.515 -0.679]] 

Time to transform: 5.8843042850494385 seconds
Chosen alpha: 1000.0
Time to fit classifier on train: 0.48134851455688477 seconds
['8' '8' '6' ... '3' '4' '6']
Time to predict: 0.006067991256713867 seconds
0.4935064935064935 accuracy for Rocket(num_kernels=5000)


In [15]:
train_and_test_ridge_ROCKETS(
    dataset="MiddlePhalanxTW",
    transformer=MiniRocket(num_kernels=10000),
    apply_augmentation=False
    )
# 0.8333333333333334 accuracy for MiniRocket(num_kernels=1000)

(399, 80, 1) train_X 

(154, 80, 1) test_X 

Before normalization:
(154, 9996) feat_test_X
[[0.25  0.875 0.162 ... 0.    0.    0.   ]
 [0.262 0.875 0.15  ... 0.    0.    0.   ]
 [0.4   0.8   0.125 ... 0.    0.    0.   ]
 ...
 [0.35  0.825 0.15  ... 0.    0.    0.   ]
 [0.387 0.837 0.112 ... 0.    0.    0.   ]
 [0.338 0.863 0.15  ... 0.    0.    0.   ]] 

After normalization:
(154, 9996) feat_test_X
[[-1.174  0.997  1.55  ...  0.     0.     0.   ]
 [-0.958  0.997  0.65  ...  0.     0.     0.   ]
 [ 1.415 -1.497 -1.151 ...  0.     0.     0.   ]
 ...
 [ 0.552 -0.666  0.65  ...  0.     0.     0.   ]
 [ 1.2   -0.25  -2.051 ...  0.     0.     0.   ]
 [ 0.336  0.581  0.65  ...  0.     0.     0.   ]] 

Time to transform: 0.29829835891723633 seconds
Chosen alpha: 1000.0
Time to fit classifier on train: 0.5569331645965576 seconds
['8' '8' '3' ... '4' '4' '3']
Time to predict: 0.00629734992980957 seconds
0.538961038961039 accuracy for MiniRocket()


In [14]:
train_and_test_ridge_ROCKETS(
    dataset="MiddlePhalanxTW",
    transformer=MultiRocket(num_kernels=1300),
    apply_augmentation=False
    )
# 0.8 accuracy for MultiRocket(num_kernels=500)

(399, 80, 1) train_X 

(154, 80, 1) test_X 

Before normalization:
(154, 10080) feat_test_X
[[ 0.262  0.812  0.162 ... 50.814  2.833 57.875]
 [ 0.275  0.812  0.175 ... 50.667  2.833 57.538]
 [ 0.412  0.712  0.162 ... 47.763  3.    51.56 ]
 ...
 [ 0.375  0.775  0.162 ... 48.5    3.    49.565]
 [ 0.387  0.8    0.15  ... 49.564  3.    54.619]
 [ 0.338  0.775  0.162 ... 50.462  3.    52.25 ]] 

After normalization:
(154, 10080) feat_test_X
[[-1.131e+00  9.240e-01 -3.513e-01 ...  1.087e+00 -1.084e+00  1.150e+00]
 [-9.342e-01  9.240e-01  4.983e-01 ...  9.589e-01 -1.084e+00  1.054e+00]
 [ 1.232e+00 -1.804e+00 -3.513e-01 ... -1.564e+00  7.693e-01 -6.549e-01]
 ...
 [ 6.411e-01 -9.915e-02 -3.513e-01 ... -9.235e-01  7.693e-01 -1.225e+00]
 [ 8.380e-01  5.830e-01 -1.201e+00 ...  1.001e-03  7.693e-01  2.195e-01]
 [ 5.034e-02 -9.915e-02 -3.513e-01 ...  7.807e-01  7.693e-01 -4.577e-01]] 

Time to transform: 0.5113236904144287 seconds
Chosen alpha: 1000.0
Time to fit classifier on train: 0.559831142425